#### import

In [4]:
a = {}
if not a:
    print(a)

{}


In [6]:
rip = 1664
gtx6 = 4456
gtx3 = 3668
rtx4 = 5205
rtx2 = 4428

s = rip*0.25 + gtx6*.7 + gtx3*0.5 + rtx4*2 + rtx2*1
s/60*250

84196.66666666667

In [1]:
import pandas as pd

from config import KEY_JSON, SHEET_INFO, PEOPLE_FIELD_MAP
from core.planner import GoogleSheetPlanner
from core.reminder import Reminder, TelegramNotifier

# import List
from typing import List, Dict
from core.repository import Task, GoogleSheetsTaskRepository
from utils.service import GoogleSheetInfo, GoogleSheetsService

from collections import defaultdict
from collections.abc import Iterable
import asyncio

import aiohttp
import json

import pytz

import nest_asyncio
nest_asyncio.apply()






In [2]:
planner = GoogleSheetPlanner(KEY_JSON, SHEET_INFO)

#planner.task_to_calendar()
#planner.designer_task_to_calendar()
#planner.task_to_table()


In [1]:
import os
from dotenv import load_dotenv
import asyncio
import json
from collections import defaultdict

import aiohttp
import pandas as pd
import pytz
from telegram.ext import Application

from utils.func import filter_stages

import nest_asyncio
nest_asyncio.apply()


load_dotenv()

TG = os.environ.get('TG_TOKEN')
OPENAI = os.environ.get('OPENAI_TOKEN')
ORG = os.environ.get('ORG_TOKEN')

TG, OPENAI, ORG

ImportError: DLL load failed while importing parsing: %1 is not a valid Win32 application.

In [ ]:
class AsyncOpenAIChatAgent(object):
    """Агент для чатов с OpenAI."""

    def __init__(self, api_key, organization=None):
        """Инициализация агента для чатов с OpenAI.

        Args:
            api_key (str): API ключ для OpenAI.
            organization (str): Название организации.
        """
        self.api_key = api_key
        self.organization = organization
        self.endpoint = 'https://api.openai.com/v1/chat/completions'

    async def chat(self, messages, model=None):
        """Чат с OpenAI GPT.

        Args:
            messages (list): Список сообщений.

        Returns:
            str: Ответ OpenAI.

        Raises:
            Exception: Если запрос к OpenAI не удался.
        """
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json',
        }
        if self.organization:
            headers['OpenAI-Organization'] = self.organization

        if isinstance(messages, str):
            messages = [{'role': 'user', 'content': messages}]

        if not model:
            model = 'gpt-3.5-turbo'
        payload = {
            'model': model,
            'messages': messages,
        }
        print(payload)
        async with aiohttp.ClientSession() as session:
            async with session.post(self.endpoint, headers=headers, data=json.dumps(payload)) as response:
                if response.status == 200:
                    data = await response.json()
                    return data['choices'][0]['message']['content']
                raise Exception(f'Failed to query OpenAI with status code: {response.status} {response}')


In [29]:
gpt = AsyncOpenAIChatAgent(OPENAI, ORG)
a = await gpt.chat('Ты кто?', model='gpt-4')
a

{'model': 'gpt-4', 'messages': [{'role': 'user', 'content': 'Ты кто?'}]}


'Я искусственный интеллект от OpenAI, называемый ChatGPT. Я был обучен на большом объеме текстовых данных и способен генерировать относительно содержательные и связные тексты на основе предложенного входного текста. Я могу помочь ответить на ваши вопросы, предложить рекомендации или просто поболтать.'

In [148]:
from telegram import Bot
bot = Bot(TG)
await bot.send_message(-4083724311, 'test')

Message(channel_chat_created=False, chat=Chat(api_kwargs={'all_members_are_administrators': True}, id=-4083724311, title='talk with ai', type=<ChatType.GROUP>), date=datetime.datetime(2023, 10, 6, 0, 5, 57, tzinfo=<UTC>), delete_chat_photo=False, from_user=User(first_name='Галя', id=6637868001, is_bot=True, username='galya_sponsor_bot'), group_chat_created=False, message_id=39, supergroup_chat_created=False, text='test')

In [138]:
from telegram import Bot
from telegram.ext import Application



In [150]:
planner = GoogleSheetPlanner(KEY_JSON, SHEET_INFO)
chat = AsyncOpenAIChatAgent(api_key=OPENAI, organization=ORG)
si = planner.sheet_info
rep = planner.task_repository
serv = planner.service
pm = PeopleManager(service=serv, sheet_info=si, )
rem = Reminder(task_repository=rep,  openai_agent=chat, helper_character=HELPER_CHARACTER, tg_bot_token=TG, people_manager=pm)

2023-10-06 03:11:23,589 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0
2023-10-06 03:11:23,594 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0


In [145]:
await rem.get_reminders()

{'Рыбкин Сергей': 'Приветик, Сереженька! Как у тебя настроение? Сегодня у нас пятница, 6 октября! 🌼\n\nУ меня для тебя небольшое напоминание о задачах на сегодня:\n1. ОЗОН // Скрикаст // для проекта «Битва Пикников» - сегодня дедлайн для сдачи "аниматика".\n\nЕсли у тебя есть какие-либо вопросы или нужна помощь, не стесняйся обратиться ко мне. Я всегда здесь, чтобы помочь! 💪🏼💕',
 'Промо Групп': 'Привет, дизайнеры! Сегодня пятница, 6 октября 😊\n\nНе забудьте, что у нас задача на сегодня:\n1. Проект "Ярче звезд" // Лендинг + конкурс // сегодня последний день для сдачи "отрисовки и согласования макетов" (25 сентября - 6 октября).\n\nУ вас все получится! Если вам нужна помощь или у вас возникли вопросы, я всегда здесь, чтобы помочь. Удачного дня! 🌟💪',
 'Карнаухов Иван': 'Привет, Иван! Как дела? Хочу напомнить тебе, что сегодня пятница, 6 октября. У нас есть несколько важных задач на сегодня:\n\n1. ОЗОН Чёрная Пятница // Динлого // для проекта «Звезды в Африке» - сегодня сдаём «префинал».\n

In [146]:
await rem.send_reminders()

In [240]:
for dis in rem.enhanced_message.keys():
    print(rem.draft_messages[dis])
    print('------------------')
    print(rem.enhanced_message[dis])
    print('------------------')
    print('//////////////////')
    print('------------------')

Привет Рыбкин Сергей! Сегодня Thursday, 05.10 в 11:07.

Задачи на Friday, 06.10:
ОЗОН // Скрикаст // для проекта «Битва Пикников» - сдаём «аниматик»

------------------
Привет, дорогой Сергей! Как твой день проходит? Хочу напомнить, что сегодня уже четверг, 5 октября. А завтра, в пятницу, 6 октября, у нас есть важная задача. Не забудь, что нужно сдать «аниматик» для проекта "Битва Пикников" в ОЗОНе. Уверена, ты сделаешь отличную работу! Если у тебя возникнут вопросы или понадобится моя помощь, я всегда здесь, чтобы поддержать тебя. Удачи с заданием! 🌟💪😊
------------------
//////////////////
------------------
Привет Баранов Артем! Сегодня Thursday, 05.10 в 11:07.

Задачи на сегодня:

Задачи на Friday, 06.10:
ТРИМЕДАТ // Дин лого до 7% // для проекта «Шоу Воли» - сдаём «Раскадровка»
ХАЛВА // Графика к устникам  // для проекта «Ярче звезд» - сдаём «съемка»

------------------
Привет, Артем! Как у тебя настроение сегодня? А я вспомнила, что сегодня четверг, 5 октября. У нас на сегодня ест

In [113]:
rem.update_date()
rem.get_actual_tasks()
today_tasks = rem.tasks['today']
next_day_tasks = rem.tasks['next_work_day']

In [114]:
rem.generate_messages(rem.dates['today'])
rem.generate_messages(rem.dates['next_work_day'])


In [147]:
agent = AsyncOpenAIChatAgent(api_key=OPENAI, organization=ORG)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"}
]
response = await agent.chat(messages)
response

'The Los Angeles Dodgers won the World Series in 2020.'

### sdf


In [31]:
705+394+546+1026+590

3261

In [98]:
event={'event_metadata': {'event_id': 'a1sgb6h6eqmun4v815cr', 'event_type': 'yandex.cloud.events.serverless.triggers.TimerMessage', 'created_at': '2023-10-05T20:30:09.953379692Z', 'cloud_id': 'b1g6d49mf4scmtn4kjki', 'folder_id': 'b1g42qj26s1u7gv7bufm'}, 'details': {'trigger_id': 'a1sldapc8v2pha7dichv'}, 'messages': [{'event_metadata': {'event_id': 'a1sgb6h6eqmun4v815cr', 'event_type': 'yandex.cloud.events.serverless.triggers.TimerMessage', 'created_at': '2023-10-05T20:30:09.953379692Z', 'cloud_id': 'b1g6d49mf4scmtn4kjki', 'folder_id': 'b1g42qj26s1u7gv7bufm'}, 'details': {'trigger_id': 'a1sldapc8v2pha7dichv', 'payload': ''}}]}

class Event:
    def __init__(self, event):
        self.event = event
        self.event_id = event['event_metadata']['event_id']
        self.event_type = event['event_metadata']['event_type']
        self.created_at = event['event_metadata']['created_at']
        self.cloud_id = event['event_metadata']['cloud_id']
        self.folder_id = event['event_metadata']['folder_id']
        self.trigger_id = event['details']['trigger_id']
        self.messages = event['messages']



Event(event_id=a1sgb6h6eqmun4v815cr, event_type=yandex.cloud.events.serverless.triggers.TimerMessage, created_at=2023-10-05T20:30:09.953379692Z, cloud_id=b1g6d49mf4scmtn4kjki, folder_id=b1g42qj26s1u7gv7bufm, trigger_id=a1sldapc8v2pha7dichv,  messages=[{'event_metadata': {'event_id': 'a1sgb6h6eqmun4v815cr', 'event_type': 'yandex.cloud.events.serverless.triggers.TimerMessage', 'created_at': '2023-10-05T20:30:09.953379692Z', 'cloud_id': 'b1g6d49mf4scmtn4kjki', 'folder_id': 'b1g42qj26s1u7gv7bufm'}, 'details': {'trigger_id': 'a1sldapc8v2pha7dichv', 'payload': ''}}])

In [1]:
import os
#TG = os.environ.get('TG_TOKEN')
#OPENAI = os.environ.get('OPENAI_TOKEN')
#ORG = os.environ.get('ORG_TOKEN')
os.environ.get('TG_TOKEN')

In [1]:
import key

print(os.environ.get('TG_TOKEN'))
from config import TG
print(TG)


NameError: name 'os' is not defined

In [2]:
from config import TG
print(TG)

6637868001:AAF-axMr_CsH0osx-QTw36-szVVYtvI7IeE


In [8]:
os.environ.get('TG_TOKEN')

'6637868001:AAF-axMr_CsH0osx-QTw36-szVVYtvI7IeE'

In [ ]:
from config import TG
TOKEN = TG  # Замените на токен вашего бота

import nest_asyncio
nest_asyncio.apply()

import logging

from telegram import ForceReply, Update
from telegram.ext import Application, CommandHandler, ContextTypes, MessageHandler, filters

from telegram import Chat, ChatMember, ChatMemberUpdated, Update
from telegram.constants import ParseMode
from telegram.ext import (
    Application,
    ChatMemberHandler,
    CommandHandler,
    ContextTypes,
    MessageHandler,
    filters,
)

# Enable logging
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
# set higher logging level for httpx to avoid all GET and POST requests being logged
logging.getLogger("httpx").setLevel(logging.WARNING)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    await update.message.reply_html(
        rf"Hi {user.mention_html()}!",
        reply_markup=ForceReply(selective=True),
    )


async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /help is issued."""
    await update.message.reply_text("Help!")


async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Echo the user message."""
    print(update)
    await context.bot.send_message(chat_id=91864013, text=str(update))
    #await update.message.reply_text(update.message.text)


def main() -> None:
    """Start the bot."""
    # Create the Application and pass it your bot's token.
    application = Application.builder().token(TOKEN).build()

    # on different commands - answer in Telegram
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))
    application.add_handler(MessageHandler(filters.UpdateFilter(), echo))

    # Run the bot until the user presses Ctrl-C
    application.run_polling(allowed_updates=Update.ALL_TYPES)


if __name__ == "__main__":
    main()

2023-10-31 17:29:05,716 - telegram.ext.Application - INFO - Application started


Update(message=Message(channel_chat_created=False, chat=Chat(id=-1001986561990, title='TNT Ivan', type=<ChatType.SUPERGROUP>), date=datetime.datetime(2023, 10, 31, 14, 31, 20, tzinfo=<UTC>), delete_chat_photo=False, entities=(MessageEntity(length=39, offset=0, type=<MessageEntityType.URL>),), from_user=User(first_name='Иван', id=363830794, is_bot=False, last_name='Карнаухов', username='i_karnaukhov'), group_chat_created=False, message_id=4770, supergroup_chat_created=False, text='https://disk.yandex.ru/d/VY0nQCKRW0ynDQ\nАльфа_лого ФИНАЛ'), update_id=504669538)


2023-10-31 17:31:18,159 - telegram.ext.Application - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "D:\PROJECTS\python\SCRIPT\TABLE_PARSE\venv\Lib\site-packages\telegram\ext\_application.py", line 1195, in process_update
    await coroutine
  File "D:\PROJECTS\python\SCRIPT\TABLE_PARSE\venv\Lib\site-packages\telegram\ext\_basehandler.py", line 153, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\solofarm\AppData\Local\Temp\ipykernel_60956\238211987.py", line 52, in echo
    await context.bot.send_message(chat_id=91864013, text=str(update))
  File "D:\PROJECTS\python\SCRIPT\TABLE_PARSE\venv\Lib\site-packages\telegram\ext\_extbot.py", line 2631, in send_message
    return await super().send_message(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\PROJECTS\python\SCRIPT\TABLE_PARSE\venv\Lib\site-packages\telegram\_bot.py", line 519, in decorator
    

Update(message=Message(channel_chat_created=False, chat=Chat(id=-1001986561990, title='TNT Ivan', type=<ChatType.SUPERGROUP>), date=datetime.datetime(2023, 10, 31, 14, 32, 54, tzinfo=<UTC>), delete_chat_photo=False, from_user=User(first_name='Daria', id=393709959, is_bot=False, is_premium=True, last_name='Vladimirovna', username='dasha_good'), group_chat_created=False, message_id=4772, message_thread_id=4770, reply_to_message=Message(channel_chat_created=False, chat=Chat(id=-1001986561990, title='TNT Ivan', type=<ChatType.SUPERGROUP>), date=datetime.datetime(2023, 10, 31, 14, 31, 20, tzinfo=<UTC>), delete_chat_photo=False, entities=(MessageEntity(length=39, offset=0, type=<MessageEntityType.URL>),), from_user=User(first_name='Иван', id=363830794, is_bot=False, last_name='Карнаухов', username='i_karnaukhov'), group_chat_created=False, message_id=4770, supergroup_chat_created=False, text='https://disk.yandex.ru/d/VY0nQCKRW0ynDQ\nАльфа_лого ФИНАЛ'), supergroup_chat_created=False, text='за

In [80]:
app = Application.builder().token(TOKEN).build()
await app.bot.send_message(chat_id=-4083724311, text='test')

Message(channel_chat_created=False, chat=Chat(api_kwargs={'all_members_are_administrators': True}, id=-4083724311, title='talk with ai', type=<ChatType.GROUP>), date=datetime.datetime(2023, 10, 5, 23, 11, 30, tzinfo=<UTC>), delete_chat_photo=False, from_user=User(first_name='Галя', id=6637868001, is_bot=True, username='galya_sponsor_bot'), group_chat_created=False, message_id=24, supergroup_chat_created=False, text='test')

In [90]:

Update(message=Message(channel_chat_created=False,
chat=Chat(api_kwargs={'all_members_are_administrators': True},
id=-4083724311,
title='talk with ai',
type=<ChatType.GROUP>),
date=datetime.datetime(2023, 10, 5, 22, 50, 27, tzinfo=<UTC>),
delete_chat_photo=False,
from_user=User(first_name='YrSolo', id=91864013, is_bot=False, is_premium=True, language_code='en', last_name='Юрий Соловьeв', username='yrsolo'),
          group_chat_created=False,
                             message_id=13,
                                        supergroup_chat_created=False,
                                                                text='asd'),
update_id=504663416)

'a1sldapc8v2pha7dichv'

### asdf